<img src="../Pierian-Data-Logo.PNG">
<br>
<strong><center>Copyright 2019. Created by Jose Marcial Portilla.</center></strong>

# What is CUDA?

Most people confuse CUDA for a language or maybe an API. It is not.

It’s more than that. CUDA is a parallel computing platform and programming model that makes using a GPU for general purpose computing simple and elegant. The developer still programs in the familiar C, C++, Fortran, or an ever expanding list of supported languages, and incorporates extensions of these languages in the form of a few basic keywords.

These keywords let the developer express massive amounts of parallelism and direct the compiler to the portion of the application that maps to the GPU.

# How do I install PyTorch for GPU?

Refer to video, its dependent on whether you have an NVIDIA GPU card or not.

# How do I know if I have CUDA available?

In [1]:
import torch
torch.cuda.is_available()
# True

False

# Using GPU and CUDA

We've provided 2 versions of our yml file, a GPU version and a CPU version. To use GPU, you need to either manually create a virtual environment, please watch the video related to this lecture, as not every computer can run GPU, you need CUDA and an NVIDIA GPU.

In [2]:
## Get Id of default device
torch.cuda.current_device()

0

In [3]:
# 0
torch.cuda.get_device_name(0) # Get name device with ID '0'

'GeForce GTX 1080 Ti'

In [4]:
# Returns the current GPU memory usage by 
# tensors in bytes for a given device
torch.cuda.memory_allocated()

0

In [5]:
# Returns the current GPU memory managed by the
# caching allocator in bytes for a given device
torch.cuda.memory_cached()

0

# Using CUDA instead of CPU

In [6]:
# CPU
a = torch.FloatTensor([1.,2.])

In [7]:
a

tensor([1., 2.])

In [8]:
a.device

device(type='cpu')

In [9]:
# GPU
a = torch.FloatTensor([1., 2.]).cuda()

In [10]:
a.device

device(type='cuda', index=0)

In [11]:
torch.cuda.memory_allocated()

512

## Sending Models to GPU

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [14]:
torch.manual_seed(32)
model = Model()

In [15]:
# From the discussions here: discuss.pytorch.org/t/how-to-check-if-model-is-on-cuda
next(model.parameters()).is_cuda

False

In [16]:
gpumodel = model.cuda()

In [17]:
next(gpumodel.parameters()).is_cuda

True

In [6]:
df = pd.read_csv('../Data/iris.csv')
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [7]:

X = df.drop('variety',axis=1).values
y = df['variety'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

In [8]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [11]:
X[0:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

## Convert Tensors to .cuda() tensors

In [22]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [ ]:
# Setting pin_memory=True: Data loader will copy tensors to CUDU (GPU) pin memory before returning them 

# trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
# testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [23]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False)

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time
epochs = 100
losses = []
start = time.time()
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

end = time.time()
print(f'TOTAL TRAINING TIME: {end-start}')

epoch:  1  loss: 1.15071142
epoch: 11  loss: 0.93773186
epoch: 21  loss: 0.77982736
epoch: 31  loss: 0.60996711
epoch: 41  loss: 0.40083539
epoch: 51  loss: 0.25436994
epoch: 61  loss: 0.15052448
epoch: 71  loss: 0.10086147
epoch: 81  loss: 0.08127660
epoch: 91  loss: 0.07230931
TOTAL TRAINING TIME: 0.4668765068054199


In [ ]:
with torch.no_grad():
    correct = 0
    for i, data in enumerate(X_test):
        y_val = gpumodel.forward(data)  
        print(f'{i+1:2}. {str(y_val):38}  {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct += 1
print(f'\n {correct} out of {len(y_test)} = {100 * correct / len(y_test):.2f}% correct')